In [1]:
import os
import logging
from pinecone import Pinecone
from langchain_pinecone import Pinecone as PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

# Load environment variables
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_INDEX_NAME = "movies-actors"

# Initialize Pinecone client
logging.info("Initializing Pinecone client...")
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

# Initialize embedding model
logging.info("Initializing embeddings model...")
embeddings_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

def retrieve_single_row(query_text: str, top_k: int = 1):
    """
    Retrieves a single row (or top-k results) from Pinecone based on a query.
    """
    # Convert query text to embedding
    query_embedding = embeddings_model.embed_query(query_text)
    
    # Search Pinecone index
    logging.info(f"Searching Pinecone for query: {query_text}")
    search_results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    # Extract and return results
    if search_results and "matches" in search_results and search_results["matches"]:
        best_match = search_results["matches"][0]  # Retrieve the highest-ranked match
        logging.info(f"Best match: {best_match}")
        return best_match
    else:
        logging.warning("No matching results found.")
        return None

# Example usage
if __name__ == "__main__":
    query = "A sci-fi movie about space travel"
    result = retrieve_single_row(query)
    print("Retrieved Row:", result)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Retrieved Row: {'id': 'tt1449385',
 'metadata': {'actor': 'Josh Craig,M. Scott Taulman,Michael G. Kaiser,Michael '
                       'Cook,Christopher R. Mihm,Mitch Obrecht',
              'actress': 'Catherine Hansen,Stephanie Mihm,Shannon '
                         'McDonough,Sid Korpi',
              'averageRating': '5.5',
              'director': 'Christopher R. Mihm',
              'endYear': '',
              'genres': 'Adventure,Sci-Fi',
              'isAdult': 'True',
              'numVotes': '84.0',
              'originalTitle': 'Destination: Outer Space',
              'primaryTitle': 'Destination: Outer Space',
              'producer': 'Josh Craig,Christopher R. Mihm',
              'runtimeMinutes': '94',
              'startYear': '2010.0',
              'tconst': 'tt1449385',
              'titleType': 'movie',
              'writer': 'Josh Craig,Christopher R. Mihm'},
 'score': 0.74891144,
 'values': []}
